In [ ]:
#"id" : App ID
#"track_name": Application name
#"size_bytes": Size in bytes
#"currency": Currency type
#"price": Price amount
#"rating_count_tot": User rating count (for the all versions)
#"rating_count_ver": User rating count (for the current version)
#"user_rating" : Average User Rating value (for the all versions)
#"user_rating_ver": Average User Rating value (for the current version)
#"ver" : Latest version code
#"cont_rating": Content rating
#"prime_genre": Primary genre
#"sup_devices.num": Number of supporting devices
#"ipadSc_urls.num": Number of screenshots showed
#"lang.num": Number of supported languages
#"vpp_lic": Vpp Device Based Licensing Enabled

In [1]:
import sys
import os
import random
from operator import add, mul
from pyspark import SparkContext, SparkConf
from pyspark import SparkFiles


cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        user_id = part.title()



app_name = '{0} : RDD_CS_1'.format(user_id)
app_name

'Edureka_960126 : RDD_CS_1'

In [2]:
conf = SparkConf().setAppName(app_name)
sc = SparkContext(conf=conf)

In [3]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/user/{0}'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

In [4]:
#1.Load csv into spark as a text file 

rdd = sc.textFile("/user/edureka_960126/AppleStore.csv")


#2. Parse the data as csv.
rdd=rdd.map(lambda line: line.split(','))
rdd=rdd.map(lambda line: [x.encode('utf-8') for x in line])

#removing the header if we want we can do this in the below fashion
rdd=rdd.zipWithIndex().filter(lambda (key,index ): (index >= 1))
rdd=rdd.map(lambda (x,y) : x)


In [5]:
#so our RDD Now looks like will be the collection of lists of individual fields as individual lines are converted into collection of fields in each record
#rdd1=rdd.map(lambda line: line[0:2] )
#rdd2=rdd.map(lambda line: line[-14:])
#rdd3=rdd.map(lambda line: line[2:len(line)-14])
rdd=rdd.map(lambda line: (line[0:2]+[','.join(line[2:len(line)-14])]+line[-14:]))
#rdd.map(lambda line: line[3] ).filter(lambda line: line != 'USD').collect()

In [7]:
#checking the results and TasK 3 is at the end---- #3 Convert bytes to MB and GB in a new column 
rdd.take(30)

[['"1"',
  '"281656475"',
  '"PAC-MAN Premium"',
  '100788224',
  '"USD"',
  '3.99',
  '21292',
  '26',
  '4',
  '4.5',
  '"6.3.5"',
  '"4+"',
  '"Games"',
  '38',
  '5',
  '10',
  '1'],
 ['"2"',
  '"281796108"',
  '"Evernote - stay organized"',
  '158578688',
  '"USD"',
  '0',
  '161065',
  '26',
  '4',
  '3.5',
  '"8.2.2"',
  '"4+"',
  '"Productivity"',
  '37',
  '5',
  '23',
  '1'],
 ['"3"',
  '"281940292"',
  '"WeatherBug - Local Weather, Radar, Maps, Alerts"',
  '100524032',
  '"USD"',
  '0',
  '188583',
  '2822',
  '3.5',
  '4.5',
  '"5.0.0"',
  '"4+"',
  '"Weather"',
  '37',
  '5',
  '3',
  '1'],
 ['"4"',
  '"282614216"',
  '"eBay: Best App to Buy, Sell, Save! Online Shopping"',
  '128512000',
  '"USD"',
  '0',
  '262241',
  '649',
  '4',
  '4.5',
  '"5.10.0"',
  '"12+"',
  '"Shopping"',
  '37',
  '5',
  '9',
  '1'],
 ['"5"',
  '"282935706"',
  '"Bible"',
  '92774400',
  '"USD"',
  '0',
  '985920',
  '5320',
  '4.5',
  '5',
  '"7.5.1"',
  '"4+"',
  '"Reference"',
  '37',
  '5',


In [9]:
#4. List top 10 trending apps 
import re

#here based on rating count like on udemy high number of users rating means trending 
#print(re.search('\"([\S\s]+)\"','"Evernote - stay organized"').group(1))

rdd.map(lambda line: [re.search('\"([\S\s]+)\"',line[2]).group(1),line[6]]).sortBy(lambda x: int(x[1]),False).take(10)






[['Facebook', '2974676'],
 ['Instagram', '2161558'],
 ['Clash of Clans', '2130805'],
 ['Temple Run', '1724546'],
 ['Pandora - Music & Radio', '1126879'],
 ['Pinterest', '1061624'],
 ['Bible', '985920'],
 ['Candy Crush Saga', '961794'],
 ['Spotify Music', '878563'],
 ['Angry Birds', '824451']]

In [10]:
#5. The difference in the average number of screenshots displayed of highest and lowest rating apps. 


#Here line[8] is the rating given to app and the line[14] is the number of screen shots
#first grouped interms of ratings and then taking the average of screen shots with respect to rating and then sorting ascending based on the rating to get the 
#highest rating average screen shot 
#when sorted in ascending then lowest rating screenshot

a=rdd.map(lambda line: [float(line[8]),int(line[14])]).groupByKey().mapValues(lambda x: sum(x) / len(x)).sortByKey(False).first()
b=rdd.map(lambda line: [float(line[8]),int(line[14])]).groupByKey().mapValues(lambda x: sum(x) / len(x)).sortByKey().first()

#average screenshots of highest rated - average ratings of lowest rated
print(a[1]-b[1])
#.reduce(lambda x,y : x+y)






1


In [11]:
#6. What percentage of high rated apps support multiple languages. 

#16th column is language and 9th column is user ratings 
#if int(line[15]) >= 2 else (0,0)
#i = 5 if a > 7 else 0

a=rdd.map(lambda line: (line[15],line[8] )).filter( lambda a : float(a[1]) >= 4  ).count()
b=rdd.map(lambda line: (line[15],line[8] )).filter( lambda a : int(a[0]) > 1 and float(a[1]) >= 4 ).count()

#note rating with graeter = 4 are considered as the high rated users
#percentage of high rated apps with rating greater than 4 with two or more languages among high rated apps with rating greater than 4
print(float(b)/float(a)*100)

#rdd.map(lambda line: line[8] ).collect()

51.7255804225


In [13]:
#7. How does app details contribute to user ratings? 
#This is an open ended question left to the wisdom of engineer 

#app details line ipadSc_urls.num which is the number of screenshots --- more screenshots showed hence more details shown to the user  

rdd.map(lambda line: [float(line[14]),float(line[8])]).groupByKey().mapValues(lambda x: sum(x) / len(x)).sortByKey(False).collect()



[(5.0, 3.814123917388408),
 (4.0, 3.4408450704225353),
 (3.0, 3.194055944055944),
 (2.0, 3.1153846153846154),
 (1.0, 3.2451612903225806),
 (0.0, 2.785147801009373)]

In [ ]:
#Hence conclusion is that more details like 5 or 4 screenshots may lead to higher ratings and 0 obviously less but 3-2-1 screenshots lead to almost similar results  

In [14]:
#8. Compare the statistics of different app groups/genres. 

#Number of apps in various categories
rdd.groupBy(lambda line: (line[12])).map(lambda (a,b) : (a,len(b))).collect()

[('"Travel"', 81),
 ('"Food & Drink"', 63),
 ('"Social Networking"', 167),
 ('"Music"', 138),
 ('"Reference"', 64),
 ('"Business"', 57),
 ('"Navigation"', 46),
 ('"Lifestyle"', 144),
 ('"Photo & Video"', 349),
 ('"Productivity"', 178),
 ('"Medical"', 23),
 ('"Games"', 3862),
 ('"Shopping"', 122),
 ('"Health & Fitness"', 180),
 ('"Catalogs"', 10),
 ('"Finance"', 104),
 ('"Weather"', 72),
 ('"Education"', 453),
 ('"News"', 75),
 ('"Sports"', 114),
 ('"Book"', 112),
 ('"Utilities"', 248),
 ('"Entertainment"', 535)]

In [15]:
#Average user rating across each genere
rdd.map(lambda line: (line[12],float(line[8]))).groupByKey().mapValues(lambda x : sum(x) / len(x) ).collect()

[('"Travel"', 3.376543209876543),
 ('"Food & Drink"', 3.1825396825396823),
 ('"Photo & Video"', 3.8008595988538683),
 ('"Music"', 3.9782608695652173),
 ('"Reference"', 3.453125),
 ('"Business"', 3.745614035087719),
 ('"Navigation"', 2.6847826086956523),
 ('"Lifestyle"', 2.8055555555555554),
 ('"Social Networking"', 2.9850299401197606),
 ('"Productivity"', 4.00561797752809),
 ('"Medical"', 3.369565217391304),
 ('"Games"', 3.6850077679958573),
 ('"Shopping"', 3.540983606557377),
 ('"Health & Fitness"', 3.7),
 ('"Catalogs"', 2.1),
 ('"Finance"', 2.4326923076923075),
 ('"Weather"', 3.5972222222222223),
 ('"News"', 2.98),
 ('"Sports"', 2.982456140350877),
 ('"Entertainment"', 3.2467289719626167),
 ('"Book"', 2.4776785714285716),
 ('"Utilities"', 3.278225806451613),
 ('"Education"', 3.376379690949227)]

In [16]:
#Average rating count across genre
rdd.map(lambda line: (line[12],int(line[6]))).groupByKey().mapValues(lambda x : sum(x) / len(x) ).collect()



[('"Travel"', 14129),
 ('"Food & Drink"', 13938),
 ('"Photo & Video"', 14352),
 ('"Music"', 28842),
 ('"Reference"', 22410),
 ('"Business"', 4788),
 ('"Navigation"', 11853),
 ('"Lifestyle"', 6161),
 ('"Social Networking"', 45498),
 ('"Productivity"', 8051),
 ('"Medical"', 592),
 ('"Games"', 13691),
 ('"Shopping"', 18615),
 ('"Health & Fitness"', 9913),
 ('"Catalogs"', 1732),
 ('"Finance"', 11047),
 ('"Weather"', 22181),
 ('"News"', 13015),
 ('"Sports"', 14026),
 ('"Entertainment"', 7533),
 ('"Book"', 5125),
 ('"Utilities"', 6863),
 ('"Education"', 2239)]

In [17]:
#9. Does length of app description contribute to the ratings? 

import re 

#here we have to load the apple store description file 
rdd_desc = sc.textFile("/user/edureka_960126/appleStore_description.csv")


rdd_desc = rdd_desc.map(lambda line : line)

#2. Parse the data as csv.
#rdd_desc=rdd_desc.map(lambda line: re.search())

#removing the header 
rdd_desc=rdd_desc.zipWithIndex().filter(lambda (key,index ): (index >= 1))
rdd_desc=rdd_desc.map(lambda (x,y) : x)


#Extracting the app Id and the size from the another rdd made from the other CSV file generated
rdd_desc = rdd_desc.map(lambda line : ((re.search('\"(\d+)\",',line).group(1),re.search(',\"(\d+)\",',line).group(1)) if (re.search('\"(\d+)\",',line) is not None) else "0"))
rdd_desc = rdd_desc.filter(lambda line : line != "0")
rdd_desc.take(5)


                                 
                                

[(u'281656475', u'100788224'),
 (u'281796108', u'158578688'),
 (u'281940292', u'100524032'),
 (u'282614216', u'128512000'),
 (u'282935706', u'92774400')]

In [18]:
rdd_desc = rdd_desc.map(lambda line: (line[0].encode('utf-8'),line[1].encode('utf-8')))

In [19]:
rdd_desc.take(2)

[('281656475', '100788224'), ('281796108', '158578688')]

In [20]:
# First creating the RDD with id and ratings from first rdd
#In this part we have joined based on App Id the two rdds created from two dataset app description and app details dataset

rdd1=rdd.map(lambda line: ((re.search('\"(\d+)\"',line[1]).group(1),float(line[8]))))



#Second rdd rdd_desc we are already having the tuples of (id,size)
new_rdd=rdd1.join(rdd_desc)

new_rdd.count()

new_rdd.sortBy(lambda line:int(line[1][1])).collect()




[('1179840697', (3.5, '1600')),
 ('840919914', (4.5, '2048')),
 ('926541293', (4.5, '589824')),
 ('771433054', (2.5, '618496')),
 ('582538300', (2.5, '671744')),
 ('544007969', (2.0, '698800')),
 ('1090653945', (2.5, '709632')),
 ('1060443742', (3.0, '746496')),
 ('962056217', (4.5, '761856')),
 ('318592730', (3.0, '767126')),
 ('445300213', (0.0, '880640')),
 ('477618130', (3.0, '885760')),
 ('794278407', (5.0, '909312')),
 ('768237041', (4.5, '962560')),
 ('1116882930', (2.0, '975872')),
 ('1071235820', (0.0, '1012736')),
 ('1093784264', (1.0, '1044480')),
 ('954323480', (4.0, '1126400')),
 ('568731681', (0.0, '1209344')),
 ('1142657936', (1.0, '1210368')),
 ('902062673', (4.5, '1259520')),
 ('1163668736', (1.5, '1327104')),
 ('870099476', (4.0, '1376256')),
 ('439495358', (4.5, '1532640')),
 ('1033645055', (1.0, '1547264')),
 ('329981776', (4.5, '1608704')),
 ('290807369', (3.5, '1646592')),
 ('1090742878', (3.0, '1700864')),
 ('1176901281', (2.0, '1734656')),
 ('998364171', (0.0, '

In [ ]:
#Above is the result showing the APPId with Tuple of (rating,size of the description ) with size in increasing order 

#as we can clearly see that as the size increase rating is not increasing or decreasing accordingly

#so we can say that the length of the description has no effect on the rating of the App 



In [21]:
#3. Convert bytes to MB and GB in a new column 

#so our size is the 4th field in the record set
#Hence we have to convert this fourth field into the GBs instead of MBs and append it to the list of record in rdd 
#which is list of list records

def col_add(line):
    x=[float(line[3])/(1024*1024)] + line
    #x.append(float(line[3])/(1024*1024))
    return x


#For addition as MB 
 #we can do it in two ways
    
#rdd=rdd.map(col_add)
rdd=rdd.map(lambda line: [int(line[3])/(1024*1024)] + line)


#For addition as GB
rdd=rdd.map(lambda line:  line+[float(line[0])/(1024)] )


#We have added two new columns for MBs and GBs in front and back just for doing something diffrent 


#Previewing the result
rdd.take(10)

[[96,
  '"1"',
  '"281656475"',
  '"PAC-MAN Premium"',
  '100788224',
  '"USD"',
  '3.99',
  '21292',
  '26',
  '4',
  '4.5',
  '"6.3.5"',
  '"4+"',
  '"Games"',
  '38',
  '5',
  '10',
  '1',
  0.09375],
 [151,
  '"2"',
  '"281796108"',
  '"Evernote - stay organized"',
  '158578688',
  '"USD"',
  '0',
  '161065',
  '26',
  '4',
  '3.5',
  '"8.2.2"',
  '"4+"',
  '"Productivity"',
  '37',
  '5',
  '23',
  '1',
  0.1474609375],
 [95,
  '"3"',
  '"281940292"',
  '"WeatherBug - Local Weather, Radar, Maps, Alerts"',
  '100524032',
  '"USD"',
  '0',
  '188583',
  '2822',
  '3.5',
  '4.5',
  '"5.0.0"',
  '"4+"',
  '"Weather"',
  '37',
  '5',
  '3',
  '1',
  0.0927734375],
 [122,
  '"4"',
  '"282614216"',
  '"eBay: Best App to Buy, Sell, Save! Online Shopping"',
  '128512000',
  '"USD"',
  '0',
  '262241',
  '649',
  '4',
  '4.5',
  '"5.10.0"',
  '"12+"',
  '"Shopping"',
  '37',
  '5',
  '9',
  '1',
  0.119140625],
 [88,
  '"5"',
  '"282935706"',
  '"Bible"',
  '92774400',
  '"USD"',
  '0',
  '

In [ ]:
#10. Create a spark-submit application for the same and print the findings in the log


#---- Jupyter Notebook has been shared instead of script to run over spark shell